In [1]:
from PIL import Image
import pandas as pd
import numpy as np 
import imageio
import os
from random import shuffle
import matplotlib.pyplot as plt

# Preprocessing

In [2]:
#checking image sizes
def get_size_statistics(DIR):
  heights = []
  widths = []
  for img in os.listdir(DIR): 
    path = os.path.join(DIR, img)
    data = np.array(Image.open(path)) #PIL Image library
    heights.append(data.shape[0])
    widths.append(data.shape[1])
  avg_height = sum(heights) / len(heights)
  avg_width = sum(widths) / len(widths)  
  print("Average Height: " + str(avg_height))
  print("Max Height: " + str(max(heights)))
  print("Min Height: " + str(min(heights)))
  print('\n')
  print("Average Width: " + str(avg_width))
  print("Max Width: " + str(max(widths)))
  print("Min Width: " + str(min(widths)))

In [3]:
get_size_statistics("/home/anthony/Documents/CodingProjects/cmpe351/Assignment2/archive/images")

Average Height: 79.99736729596543
Max Height: 80
Min Height: 60


Average Width: 59.99815485700142
Max Width: 60
Min Width: 53


The first step is to format the image data so that we have easy inputs for our keras model. We will create a dictionary of the training data to map to the id of each jpg. The same process will also be repeated with the testing data.

In [4]:
#dictionary mapping training id to the item categories
train_id_dict = {} 
training_f= open("train.csv", "r")
training_contents = training_f.read()
training_contents = training_contents.split('\n')
for i in range(len(training_contents)-1):
    training_contents[i] = training_contents[i].split('\t')
    train_id_dict[training_contents[i][0]] = training_contents[i][1]

train_id_dict.pop("imageid")

'label'

In [5]:
#dictionary mapping test id to the item categories
test_id_dict = {} 
test_f= open("test.csv", "r")
test_contents = test_f.read()
test_contents = test_contents.split('\n')
for i in range(len(test_contents)-1):
    test_contents[i] = test_contents[i].split('\t')
    test_id_dict[test_contents[i][0]] = test_contents[i][1]

test_id_dict.pop("imageid")

'label'

In [6]:
from keras.preprocessing import image

2022-03-09 23:22:47.674427: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-09 23:22:47.674448: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
#create dictionary for the categories so that images can be renamed
train_categories = train_id_dict.values()
train_set_categories = set(train_categories)
train_counting_dict = {}
for i in train_set_categories:
  train_counting_dict[i] = 0

test_categories = test_id_dict.values()
test_set_categories = set(test_categories)
test_counting_dict = {}
for i in test_set_categories:
  test_counting_dict[i] = 0

In [8]:
#for splitting the image dataset into training and validation
train_images = []
test_images = []

for img in os.listdir('/home/anthony/Documents/CodingProjects/cmpe351/Assignment2/archive/images'):
    #check if the image id matches the image id in training id dict
    imgName = img.split('.')[0]
    
    #if the image id is in the training dictionary create a folder of labeled training images 
    if imgName in train_id_dict.keys():
        train_label = train_id_dict[str(imgName)]
        train_counting_dict[train_label] += 1
        path = os.path.join('/home/anthony/Documents/CodingProjects/cmpe351/Assignment2/archive/images', img)
        saveName = './labeled_train/' + train_label + '-' + str(train_counting_dict[train_label]) + '.jpg'
        train_image_data = np.array(Image.open(path))
        imageio.imwrite(saveName, train_image_data)
    
    #if the image id is in the testing dictionary create a folder of labeled training images 
    else:
        test_label = test_id_dict[str(imgName)]
        test_counting_dict[test_label] += 1 
        path = os.path.join('/home/anthony/Documents/CodingProjects/cmpe351/Assignment2/archive/images', img)
        saveName = './labeled_test/' + test_label + '-' + str(test_counting_dict[test_label]) + '.jpg'
        test_image_data = np.array(Image.open(path))
        imageio.imwrite(saveName, test_image_data)

In [9]:
#one-hot-encoding target variables 
def label_img(name):
    label = name.split('-')[0]
    if label == 'Watches':
        return np.array([1,0,0,0,0,0,0,0,0,0,0,0,0])
    elif label == 'Innerwear':
        return np.array([0,1,0,0,0,0,0,0,0,0,0,0,0])
    elif label == 'Bags':
        return np.array([0,0,1,0,0,0,0,0,0,0,0,0,0])
    elif label == 'Bottomwear':
        return np.array([0,0,0,1,0,0,0,0,0,0,0,0,0])  
    elif label == 'Shoes':
        return np.array([0,0,0,0,1,0,0,0,0,0,0,0,0]) 
    elif label == 'Fragrance':
        return np.array([0,0,0,0,0,1,0,0,0,0,0,0,0])
    elif label == 'Jewellery':
        return np.array([0,0,0,0,0,0,1,0,0,0,0,0,0])
    elif label == 'Sandal':
        return np.array([0,0,0,0,0,0,0,1,0,0,0,0,0])
    elif label == 'Topwear':
        return np.array([0,0,0,0,0,0,0,0,1,0,0,0,0])
    elif label == 'Wallets':
        return np.array([0,0,0,0,0,0,0,0,0,1,0,0,0])
    elif label == 'Makeup':
        return np.array([0,0,0,0,0,0,0,0,0,0,1,0,0])
    elif label == 'Others':
        return np.array([0,0,0,0,0,0,0,0,0,0,0,1,0])
    elif label == 'Eyewear':
        return np.array([0,0,0,0,0,0,0,0,0,0,0,0,1])    

Images should be formatted for best results. Also a basic data augmentation technique is used in hopes of better  performance.

In [10]:
#Load and resize images
IMG_SIZE = 80

#format images to all be the same size
def format_images(directory):
  data = []
  for img in os.listdir(directory):
    label = label_img(img)
    path = os.path.join(directory, img)
    img = Image.open(path)
    img = img.convert('L')
    img = img.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
    data.append([np.array(img), label])    
    
    # Basic Data Augmentation - Horizontal Flipping
    flip_img = Image.open(path)
    flip_img = flip_img.convert('L')
    flip_img = flip_img.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
    flip_img = np.array(flip_img)
    flip_img = np.fliplr(flip_img)
    data.append([flip_img, label])  
    shuffle(data)
  return data

In [11]:
#checking that everyhing worked
train_data = format_images("/home/anthony/Documents/CodingProjects/cmpe351/Assignment2/labeled_train")
test_data = format_images("/home/anthony/Documents/CodingProjects/cmpe351/Assignment2/labeled_test")

# Part 1 - Simple CNN For Image Classification

In part 1 we will use keras to design a CNN model.

In [12]:
#keras imports 
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, BatchNormalization, Dropout
from tensorflow.keras.utils import to_categorical

In [13]:
#setting up training data
train_images = np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
train_labels = np.array([i[1] for i in train_data])

In [14]:
#CNN Creation
model = Sequential()

#adding layers to the model
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(13, activation = 'softmax'))

2022-03-10 13:22:41.187470: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-10 13:22:41.188253: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-10 13:22:41.190205: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (anthony): /proc/driver/nvidia/version does not exist
2022-03-10 13:22:41.202285: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#train the model
model.fit(train_images, train_labels, epochs=5, verbose=1)

In [ ]:
model.summary()

Error: Session cannot generate requests

In [ ]:
#setting up testing data
test_images = np.array([i[0] for i in test_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_labels = np.array([i[1] for i in test_data])

In [ ]:
#evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose = 0)
print(acc * 100)

Error: Session cannot generate requests

# Part 2 - Upgrading the Model

Too upgrade the model I'm going to change the parameters and check how the impact. I added 3 dropout layers these are to help with the issue of overfitting. wanted to add more epochs but my computer could not handle it

In [ ]:
#CNN Creation
improved_model = Sequential()

#adding layers to the improved_model
improved_model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
improved_model.add(MaxPooling2D(pool_size=(2,2)))
improved_model.add(BatchNormalization())

improved_model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
improved_model.add(MaxPooling2D(pool_size=(2,2)))
improved_model.add(BatchNormalization())

improved_model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
improved_model.add(MaxPooling2D(pool_size=(2,2)))
improved_model.add(BatchNormalization())

improved_model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
improved_model.add(MaxPooling2D(pool_size=(2,2)))
improved_model.add(BatchNormalization())

improved_model.add(Flatten())
improved_model.add(Dense(256, activation='relu'))
improved_model.add(Dropout(0.2))
improved_model.add(Dense(128, activation='relu'))
improved_model.add(Dropout(0.3))
improved_model.add(Dense(13, activation = 'softmax'))

In [ ]:
#compile model using accuracy to measure model performance
improved_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#train the model
improved_model.fit(train_images, train_labels, epochs=5, verbose=1)

In [ ]:
improved_model.summary()

Error: Session cannot generate requests

In [ ]:
loss, acc = improved_model.evaluate(test_images, test_labels, verbose = 0)
print(acc * 100)

Error: Session cannot generate requests